# fetch_data.py functions

These functions will allow the user to key in their selected cryptocurrencies in their portfolio, and retrieve historical prices from yfinance, a Yahoo! Finance API.

To use these functions in the main script, `import fetch_data.py as fd`.

Functions: `fd.input_crypto()`, `fd.get_ticker_data(ticker_list)`

---


## Import libraries and dependencies

In [2]:
import pandas as pd

import datetime

# Import quantstats
import quantstats as qs
qs.extend_pandas()
# Import yfinance
import yfinance as yf


## Function to request user input for the cryptocurrencies they have, and the number of years back for which data is to be fetched.

In [17]:
def get_portfolio_data():
    '''
    Requests user to key in crypto tickers, and appends to a list.
    '''
    ticker_list = []
    ticker = ""
    print("Please enter the tickers of your cryptocurrencies one by one")
    print("Type 'done' when finished.")
    
    # Keep requesting input from user if user has not typed 'done'
    while ticker.casefold() != "done":
        ticker = str.upper(input("Ticker: "))                           
        if ticker.casefold() != "done":
            ticker_list.append(ticker)
        else:
            break
    
    # Adds "-USD" suffix to each crypto ticker for compatibility with yfinance
    ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
    return ticker_list

def get_investment_amt():
    '''
    Requests investment amount from user. 
    '''
    investment_amount = ""
    while type(investment_amount) is not float:
        try:
            investment_amount = float(input("How much do you wish to invest in total?\n"
                                            "(Please input amount without currency symbol)\n"))
        except Exception:
            print('Error: Please input a numerical value\n'
                 'Remember not to include the currency symbol\n')    
    return investment_amount

        
ticker_list = input_cryptos() 
investment_amt = get_investment_amt()

print(f"--------------------------")                      
print(f"Cryptocurrencies selected:")
print(*ticker_list, sep=', ')
print(f"--------------------------")                     
print(f"Investment amount:")
print(f"${investment_amount:.2f}")
print(f"--------------------------")  

                        

Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  eth
Ticker:  luna
Ticker:  btc
Ticker:  done


--------------------------
Cryptocurrencies selected:
ETH, LUNA, BTC
--------------------------


How much do you wish to invest in total?
(Please input amount without currency symbol)
 2000


--------------------------
Investment amount:
$2000.00
--------------------------


In [21]:
# Function to get ticker data. years_back set to 3 by default, given most altcoins don't have a price history dating back more than that. 
def get_ticker_data(ticker_list, years_back = 3):
    ''''
    Iterates through each ticker in user portfolio and fetches OHLCV data from Yahoo Finance API into a pandas dataframe. 
    Also records daily returns for each cryptocurrency as separate columns within the same dataframe.

    Parameters:
    ticker_list (list): A list of cryptocurrency tickers for which data is to be fetched
    years_back (int): Number of years back from the current date, for which data is to be fetched. Set to 3 by default
    '''
    # Initialise dict to record OHLCV data
    d ={}
    # Define start and end dates
    end = datetime.datetime.now()
    start = datetime.datetime.now() - datetime.timedelta(days= 365 * years_back)
    # create empty dataframe
    cryptos_final = pd.DataFrame()
    
    # Get timeframe 
    interval_list = ['1d', '5d', '1wk', '1mo', '3mo']
    interval = str(input(
        f"\nPlease select the interval: \n"
        f"(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)\n"
    )
                  )
                  
    while interval not in interval_list:
        print("\nError: You have selected an invalid interval")
        interval = str(input(
            f"Please select the interval: \n"
            f"(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)\n"
        )
                      )
                              
    for ticker in ticker_list:
        try:
            # download the crypto price 
            crypto = yf.Ticker(ticker)
            crypto_df = crypto.history(
                start = start, 
                end = end, 
                interval = interval
            )
            
            # append the individual crpyto prices 
            if len(crypto_df) == 0:
                None
            else:
                d[(ticker, "open")] = crypto_df['Open']
                d[(ticker, "high")] = crypto_df['High']
                d[(ticker, "low")] = crypto_df["Low"]
                d[(ticker, "close")] = crypto_df["Close"]
                d[(ticker, "volume")] = crypto_df["Volume"]
                d[(ticker, "daily_return")] = crypto_df['Close'].pct_change()
        except Exception:
            None
    d = pd.DataFrame(d)
    d.dropna(inplace = True)
    return d

get_ticker_data(ticker_list)



Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)
 2d



Error: You have selected an invalid interval


Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo) 1d


ETH-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2019-01-11   127.813965   130.165939   125.244942   127.548325   2667585234   
2019-01-12   127.528084   128.666122   125.446754   125.966530   2212109224   
2019-01-13   125.907227   126.267876   116.085968   116.897804   2268263944   
2019-01-14   116.979095   130.856735   116.967560   129.068726   2798085735   
2019-01-15   129.172684   130.743561   120.311523   122.032715   2735449164   
...                 ...          ...          ...          ...          ...   
2022-01-04  3761.361572  3876.785156  3723.349854  3794.056641  14030925983   
2022-01-05  3794.269043  3842.059814  3456.745361  3550.386963  18061338502   
2022-01-06  3549.708984  3549.708984  3335.376709  3418.408203  24293791313   
2022-01-07  3417.837891  3420.461914  3117.381348  3193.210449  24494179209   
2022-01-08  3193.502441  3240.304443  3020.880859  3091.972656  16037732700   

                         LUNA-USD                                       \
           daily_return      open      high       low     close volume   
Date                                                                     
2019-01-11    -0.008372  0.000844  0.001062  0.000796  0.000804    8.0   
2019-01-12    -0.012402  0.000803  0.000822  0.000782  0.000819   13.0   
2019-01-13    -0.071993  0.000818  0.001305  0.000674  0.000676   46.0   
2019-01-14     0.104116  0.000677  0.001715  0.000675  0.001702    7.0   
2019-01-15    -0.054514  0.001701  0.001709  0.000789  0.000824    6.0   
...                 ...       ...       ...       ...       ...    ...   
2022-01-04     0.008687  0.008827  0.009007  0.008693  0.008721   91.0   
2022-01-05    -0.064224  0.008721  0.009345  0.008560  0.008714   56.0   
2022-01-06    -0.037173  0.008713  0.009085  0.008529  0.009064  649.0   
2022-01-07    -0.065878  0.009062  0.009062  0.008215  0.008312  124.0   
2022-01-08    -0.031704  0.008312  0.008437  0.008134  0.008347    0.0   

                              BTC-USD                              \
           daily_return          open          high           low   
Date                                                                
2019-01-11    -0.047393   3674.015381   3713.881836   3653.069824   
2019-01-12     0.018657   3686.973145   3698.978271   3653.810791   
2019-01-13    -0.174603   3658.868164   3674.760010   3544.927246   
2019-01-14     1.517751   3557.311035   3727.836182   3552.285156   
2019-01-15    -0.515864   3704.216309   3720.153320   3619.949219   
...                 ...           ...           ...           ...   
2022-01-04    -0.012009  46458.851562  47406.546875  45752.464844   
2022-01-05    -0.000803  45899.359375  46929.046875  42798.222656   
2022-01-06     0.040165  43565.511719  43748.718750  42645.539062   
2022-01-07    -0.082966  43153.570312  43153.570312  41077.445312   
2022-01-08     0.004211  41561.464844  42228.941406  40672.277344   

                                                    
                   close       volume daily_return  
Date                                                
2019-01-11   3687.365479   5538712865     0.002294  
2019-01-12   3661.301025   4778170883    -0.007069  
2019-01-13   3552.953125   4681302466    -0.029593  
2019-01-14   3706.052246   5651384490     0.043091  
2019-01-15   3630.675293   5537192302    -0.020339  
...                  ...          ...          ...  
2022-01-04  45897.574219  42494677905    -0.012066  
2022-01-05  43569.003906  36851084859    -0.050734  
2022-01-06  43160.929688  30208048289    -0.009366  
2022-01-07  41557.902344  84196607520    -0.037141  
2022-01-08  41733.941406  28066355845     0.004236  

[1094 rows x 18 columns]